In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import seaborn as sns
sns.set(style='darkgrid')

import sys
sys.path.append('../src') 

from utils import (stock_trading_strategy,
                   stock_trading_strategy_backwards,
                   stock_trading_strategy_finer,
                   plot_trading_strategy,
                   plot_trading_strategy_finer,
                   download_stock_df,
                   download_stock_df_full,
                   user_function,
                   plot_candlestick,
                  )

In [2]:
def calculate_investment_return(stock_prices, purchase_interval, investment_amount, start_date, end_date):
    # Filter stock prices between start and end date
    stock_prices = stock_prices[stock_prices['date'].between(start_date,end_date)]
    

    # Initialize variables
    total_investment = 0
    num_shares = 0
    daily_investment_value = list()
    daily_investment_amt = list()    
    last_purchase_date = None

    # Iterate over stock prices
    for idx, row in stock_prices.iterrows():
        # Check if it's time to make a purchase
        if last_purchase_date is None or (row['date'] - last_purchase_date).days >= purchase_interval:
            # Calculate number of shares to purchase
            num_shares += investment_amount / row['price']
            total_investment += investment_amount      
            last_purchase_date = row['date']
        else:
            num_shares += 0
            total_investment += 0
        
        # Calculate daily investment value
        daily_investment_value.append(num_shares * row['price'])
        daily_investment_amt.append(total_investment)

    df_strategy = stock_prices.copy()
    df_strategy['investment_value'] = daily_investment_value
    df_strategy['investment_amt_sofar'] = daily_investment_amt
    df_strategy['revenue'] = df_strategy['investment_value'] - df_strategy['investment_amt_sofar']
    df_strategy['revenue%'] = df_strategy['revenue'] / df_strategy['investment_amt_sofar']
    return df_strategy

# Example usage
# Assume stock_prices is a pandas dataframe with 'Date' as index and 'Price' as column
# Replace this with your actual data

purchase_interval = 7  # Weekly purchase
investment_amount = 1000  # $1000 per purchase
start_date = '2024-01-01'
end_date = '2024-01-31'

purchase_interval = 7  # Weekly purchase
investment_amount = 1000  # $1000 per purchase
start_date = '2024-01-01'
end_date = '2024-01-31'
stock_prices = pd.DataFrame({
    'date': pd.date_range(start='2024-01-01', periods=7, freq='D'),
    'price': [100, 110, 105, 120, 115, 125, 130],
})

stock_strategy =  calculate_investment_return(stock_prices, purchase_interval, investment_amount, start_date, end_date)
stock_strategy

,date,price,investment_value,investment_amt_sofar,revenue,revenue%
0,2024-01-01,100,1000.0,1000,0.0,0.00
1,2024-01-02,110,1100.0,1000,100.0,0.10
2,2024-01-03,105,1050.0,1000,50.0,0.05
3,2024-01-04,120,1200.0,1000,200.0,0.20
4,2024-01-05,115,1150.0,1000,150.0,0.15
5,2024-01-06,125,1250.0,1000,250.0,0.25
6,2024-01-07,130,1300.0,1000,300.0,0.30


In [4]:
df_brk = download_stock_df_full('brk-b')
stock_strategy =  calculate_investment_return(df_brk[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])

stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.16435534399307336
1550    0.264095
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,300.790009,500.000000,500,0.000000,0.000000
1009,2022-01-04,308.529999,512.866103,500,12.866103,0.025732
1010,2022-01-05,309.920013,515.176709,500,15.176709,0.030353
1011,2022-01-06,313.220001,520.662243,500,20.662243,0.041324
1012,2022-01-07,319.779999,531.566857,500,31.566857,0.063134
...,...,...,...,...,...,...
1546,2024-02-26,409.140015,71281.140421,56000,15281.140421,0.272878
1547,2024-02-27,408.910004,71241.067476,56000,15241.067476,0.272162
1548,2024-02-28,412.140015,72303.806046,56500,15803.806046,0.279713
1549,2024-02-29,409.399994,71823.110355,56500,15323.110355,0.271205


In [5]:
df_voo = download_stock_df_full('voo')
stock_strategy =  calculate_investment_return(df_voo[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])
stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.14854136518103192
1550    0.217705
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,439.250000,500.000000,500,0.000000,0.000000
1009,2022-01-04,439.070007,499.795114,500,-0.204886,-0.000410
1010,2022-01-05,430.660004,490.221973,500,-9.778027,-0.019556
1011,2022-01-06,430.079987,489.561738,500,-10.438262,-0.020877
1012,2022-01-07,428.589996,487.865676,500,-12.134324,-0.024269
...,...,...,...,...,...,...
1546,2024-02-26,465.070007,67372.332944,56000,11372.332944,0.203077
1547,2024-02-27,465.929993,67496.914660,56000,11496.914660,0.205302
1548,2024-02-28,465.209991,67892.611735,56500,11392.611735,0.201639
1549,2024-02-29,466.929993,68143.628216,56500,11643.628216,0.206082


In [62]:
df_qqq =  download_stock_df_full('qqq')
stock_strategy =  calculate_investment_return(df_qqq[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])
stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.18730736363198602
1548    0.344876
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,401.679993,500.000000,500,5.684342e-14,1.136868e-16
1009,2022-01-04,396.470001,993.514749,1000,-6.485251e+00,-6.485251e-03
1010,2022-01-05,384.290009,1462.992887,1500,-3.700711e+01,-2.467141e-02
1011,2022-01-06,384.019989,1961.964922,2000,-3.803508e+01,-1.901754e-02
1012,2022-01-07,379.859985,2440.711390,2500,-5.928861e+01,-2.371544e-02
...,...,...,...,...,...,...
1544,2024-02-22,438.070007,339735.475076,251000,8.873548e+04,3.535278e-01
1545,2024-02-23,436.779999,338735.037569,251000,8.773504e+04,3.495420e-01
1546,2024-02-26,436.549988,338556.657652,251000,8.755666e+04,3.488313e-01
1547,2024-02-27,437.600006,339370.976056,251000,8.837098e+04,3.520756e-01


### Compare to all-in

In [46]:
df_brk = download_stock_df_full('brk-b')
start_date = []
start_price = []
all_in_return = []
fix_invest_return = []
for idx, row in df_brk[df_brk['date'].between('2020-03-01', '2023-03-01')].iterrows():
    
    all_in_return.append((df_brk.iloc[idx+252]['close'] - df_brk.iloc[idx]['close']) / df_brk.iloc[idx]['close'])
    stock_strategy =  calculate_investment_return(df_brk[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date=df_brk.iloc[idx]['date'], 
                                              end_date=df_brk.iloc[idx+252]['date'])
    fix_invest_return.append(stock_strategy.tail(1)['revenue%'].values[0])
    start_date.append(row['date'])
    start_price.append(row['close'])

print(len(fix_invest_return), len(all_in_return))
print(np.mean(all_in_return), np.std(all_in_return))
print(np.mean(fix_invest_return), np.std(fix_invest_return))
print((np.array(all_in_return) > np.array(fix_invest_return)).sum()/len(fix_invest_return))

[*********************100%%**********************]  1 of 1 completed
756 756
0.20422727297087528 0.18191826313803786
0.09675283995315773 0.0941910821561216
0.8492063492063492


In [47]:
df_check = pd.DataFrame({
    'start_date': start_date,
    'start_price': start_price,
    'all_in_return': all_in_return,
    'fix_return': fix_invest_return,
})
df_check

,start_date,start_price,all_in_return,fix_return
0,2020-03-02,217.630005,0.145155,0.214685
1,2020-03-03,208.809998,0.204684,0.223077
2,2020-03-04,217.860001,0.126779,0.195193
3,2020-03-05,208.960007,0.211476,0.239599
4,2020-03-06,205.979996,0.250655,0.252855
...,...,...,...,...
751,2023-02-23,303.070007,0.349985,0.189218
752,2023-02-24,304.019989,0.345010,0.187661
753,2023-02-27,304.660004,0.352787,0.191913
754,2023-02-28,305.179993,0.341503,0.184849


In [48]:
df_voo = download_stock_df_full('voo')
start_date = []
start_price = []
all_in_return = []
fix_invest_return = []
for idx, row in df_voo[df_brk['date'].between('2020-03-01', '2023-03-01')].iterrows():
    
    all_in_return.append((df_voo.iloc[idx+252]['close'] - df_voo.iloc[idx]['close']) / df_voo.iloc[idx]['close'])
    stock_strategy =  calculate_investment_return(df_voo[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date=df_voo.iloc[idx]['date'], 
                                              end_date=df_voo.iloc[idx+252]['date'])
    fix_invest_return.append(stock_strategy.tail(1)['revenue%'].values[0])
    start_date.append(row['date'])
    start_price.append(row['close'])

print(len(fix_invest_return), len(all_in_return))
print(np.mean(all_in_return), np.std(all_in_return))
print(np.mean(fix_invest_return), np.std(fix_invest_return))
print((np.array(all_in_return) > np.array(fix_invest_return)).sum()/len(fix_invest_return))

[*********************100%%**********************]  1 of 1 completed
756 756
0.12207909576088598 0.19930849221380306
0.05261112541698369 0.09565975969201666
0.6785714285714286


In [49]:
df_qqq = download_stock_df_full('qqq')
start_date = []
start_price = []
all_in_return = []
fix_invest_return = []
for idx, row in df_qqq[df_brk['date'].between('2020-03-01', '2023-03-01')].iterrows():
    
    all_in_return.append((df_qqq.iloc[idx+252]['close'] - df_qqq.iloc[idx]['close']) / df_qqq.iloc[idx]['close'])
    stock_strategy =  calculate_investment_return(df_qqq[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date=df_qqq.iloc[idx]['date'], 
                                              end_date=df_qqq.iloc[idx+252]['date'])
    fix_invest_return.append(stock_strategy.tail(1)['revenue%'].values[0])
    start_date.append(row['date'])
    start_price.append(row['close'])

print(len(fix_invest_return), len(all_in_return))
print(np.mean(all_in_return), np.std(all_in_return))
print(np.mean(fix_invest_return), np.std(fix_invest_return))
print((np.array(all_in_return) > np.array(fix_invest_return)).sum()/len(fix_invest_return))

[*********************100%%**********************]  1 of 1 completed
756 756
0.1524021520064968 0.2795820508650246
0.06547365726200009 0.1383328822825879
0.6613756613756614


In [50]:
df_tqqq = download_stock_df_full('tqqq')
start_date = []
start_price = []
all_in_return = []
fix_invest_return = []
for idx, row in df_tqqq[df_brk['date'].between('2020-03-01', '2023-03-01')].iterrows():
    
    all_in_return.append((df_tqqq.iloc[idx+252]['close'] - df_tqqq.iloc[idx]['close']) / df_tqqq.iloc[idx]['close'])
    stock_strategy =  calculate_investment_return(df_tqqq[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date=df_tqqq.iloc[idx]['date'], 
                                              end_date=df_tqqq.iloc[idx+252]['date'])
    fix_invest_return.append(stock_strategy.tail(1)['revenue%'].values[0])
    start_date.append(row['date'])
    start_price.append(row['close'])

print(len(fix_invest_return), len(all_in_return))
print(np.mean(all_in_return), np.std(all_in_return))
print(np.mean(fix_invest_return), np.std(fix_invest_return))
print((np.array(all_in_return) > np.array(fix_invest_return)).sum()/len(fix_invest_return))

[*********************100%%**********************]  1 of 1 completed
756 756
0.4659002444292506 0.9785201512333649
0.17361111060913195 0.42246291620569026
0.5978835978835979


In [3]:
def compare_fix_allin(stock_df):
    all_in_return = []
    fix_invest_return = []
    for idx, row in stock_df[stock_df['date'].between('2022-03-01', '2023-03-01')].iterrows():
        
        all_in_return.append((stock_df.iloc[idx+252]['close'] - stock_df.iloc[idx]['close']) / stock_df.iloc[idx]['close'])
        stock_strategy =  calculate_investment_return(stock_df[['date','close']].rename(columns={'close':'price'}),
                                                  purchase_interval=7, 
                                                  investment_amount=500, 
                                                  start_date=stock_df.iloc[idx]['date'], 
                                                  end_date=stock_df.iloc[idx+252]['date'])
        fix_invest_return.append(stock_strategy.tail(1)['revenue%'].values[0])
    
    print(len(fix_invest_return), len(all_in_return))
    print(np.mean(all_in_return), np.std(all_in_return))
    print(np.mean(fix_invest_return), np.std(fix_invest_return))
    print((np.array(all_in_return) > np.array(fix_invest_return)).sum()/252)
    return (np.array(all_in_return) > np.array(fix_invest_return)).sum()/252

In [8]:
df_brk = download_stock_df_full('brk-b')
compare_fix_allin(df_brk)

[*********************100%%**********************]  1 of 1 completed
252 252
0.1526560393231015 0.1327481715645491
0.10040908520567346 0.04782216281617542
0.7182539682539683


0.7182539682539683

In [7]:
df_voo = download_stock_df_full('voo')
compare_fix_allin(df_voo)

[*********************100%%**********************]  1 of 1 completed
252 252
0.1058205993417433 0.10577529548404033
0.07768662746150438 0.045189049216015256
0.6547619047619048


0.6547619047619048

In [6]:
df_qqq = download_stock_df_full('qqq')
compare_fix_allin(df_qqq)

[*********************100%%**********************]  1 of 1 completed
252 252
0.24020978639053683 0.1931743148216476
0.16333728040981307 0.06462960817671859
0.6587301587301587


0.6587301587301587

In [5]:
df_tqqq = download_stock_df_full('tqqq')
compare_fix_allin(df_tqqq)

[*********************100%%**********************]  1 of 1 completed
252 252
0.5797497851555246 0.7114017560540337
0.43279357738276003 0.23845168133826222
0.49206349206349204


0.49206349206349204

In [4]:
# df_upro = download_stock_df_full("upro")
# compare_fix_allin(df_upro)

[*********************100%%**********************]  1 of 1 completed
252 252
0.17854987254257887 0.3337985876990489
0.16985319056983936 0.15075793050315192
0.5515873015873016


0.5515873015873016

In [10]:
df_spxl = download_stock_df_full("SPXL")
compare_fix_allin(df_spxl)

[*********************100%%**********************]  1 of 1 completed
252 252
0.18567820943670926 0.33316446156677826
0.17274337067347306 0.1498782292764072
0.5515873015873016


0.5515873015873016

In [11]:
df_sso = download_stock_df_full("SSO")
compare_fix_allin(df_sso)

[*********************100%%**********************]  1 of 1 completed
252 252
0.1601214910713252 0.2184199351994994
0.13123364325829318 0.09549845540069053
0.5992063492063492


0.5992063492063492

In [12]:
# df_spuu = download_stock_df_full("SPUU")
# compare_fix_allin(df_spuu)

[*********************100%%**********************]  1 of 1 completed
252 252
0.15781687816547615 0.21633725196267997
0.12944856753034964 0.09479615498065096
0.6031746031746031


0.6031746031746031

### BRK is the god! Check correlation

In [13]:
print('BRK and VOO:', np.corrcoef(df_brk['close'], df_voo['close'])[0, 1])
print('BRK and SPXL:', np.corrcoef(df_brk['close'], df_spxl['close'])[0, 1])

print('BRK and QQQ:', np.corrcoef(df_brk['close'], df_qqq['close'])[0, 1])
print('BRK and TQQQ:', np.corrcoef(df_brk['close'], df_tqqq['close'])[0, 1])

BRK and VOO: 0.9148002245262692
BRK and SPXL: 0.7730117420565301
BRK and QQQ: 0.8598661044770971
BRK and TQQQ: 0.6042651056811051
